## 1. Setup & Data Loading

We importeren de nodige libraries voor data analyse (pandas, numpy), visualisatie (matplotlib, seaborn) en tijdreeksverwerking. De plotting settings zorgen voor consistente en professionele grafieken doorheen de hele notebook.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [2]:
from pathlib import Path
import re

# Pas pad aan indien nodig
data_dir = Path('../data/demand')
if not data_dir.exists():
    data_dir = Path('../data/')
if not data_dir.exists():
    data_dir = Path('../data/demand/')

csv_files = sorted(data_dir.glob('*.csv'))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {data_dir.resolve()}")

loaded = []
print("✅ Files found:")
for f in csv_files:
    df_tmp = pd.read_csv(f)
    loaded.append((f.name, df_tmp))
    # Probeer jaartal uit bestandsnaam te halen en maak variable df_YYYY
    m = re.search(r'(\d{4})', f.stem)
    if m:
        year = m.group(1)
        var_name = f"df_{year}"
        globals()[var_name] = df_tmp
        print(f"  - {f.name} -> variable {var_name}: {len(df_tmp)} rows")
    else:
        print(f"  - {f.name}: {len(df_tmp)} rows (no year in filename)")

# Samengevoegd dataframe (gebruik df_raw of vervang later bestaande concat)
df_raw = pd.concat([t[1] for t in loaded], ignore_index=True)
print(f"\n✅ Total combined rows: {len(df_raw)}")

✅ Files found:
  - demanddata_2001.csv -> variable df_2001: 17520 rows
  - demanddata_2002.csv -> variable df_2002: 17520 rows
  - demanddata_2003.csv -> variable df_2003: 17520 rows
  - demanddata_2004.csv -> variable df_2004: 17568 rows
  - demanddata_2005.csv -> variable df_2005: 17520 rows
  - demanddata_2006.csv -> variable df_2006: 17520 rows
  - demanddata_2007.csv -> variable df_2007: 17520 rows
  - demanddata_2008.csv -> variable df_2008: 17568 rows
  - demanddata_2009.csv -> variable df_2009: 17520 rows
  - demanddata_2010.csv -> variable df_2010: 17520 rows
  - demanddata_2011.csv -> variable df_2011: 17520 rows
  - demanddata_2012.csv -> variable df_2012: 17568 rows
  - demanddata_2013.csv -> variable df_2013: 17520 rows
  - demanddata_2014.csv -> variable df_2014: 17520 rows
  - demanddata_2015.csv -> variable df_2015: 17520 rows
  - demanddata_2016.csv -> variable df_2016: 17568 rows
  - demanddata_2017.csv -> variable df_2017: 17520 rows
  - demanddata_2018.csv -> variab

We laden alle jaarlijkse CSV-bestanden in vanuit de data directory. Elk bestand bevat half-uurlijkse metingen van elektriciteitsverbruik en gerelateerde variabelen. Let op dat schrikkeljaren (2004, 2008, 2012, 2016, 2020, 2024) 17568 rijen hebben in plaats van 17520 omdat deze jaren een extra dag bevatten (366 dagen × 48 metingen per dag). Het jaar 2025 bevat slechts 13246 rijen omdat we enkel data tot begin oktober hebben. In totaal hebben we 434014 datapunten over een periode van bijna 25 jaar (2001-2025).

In [3]:
import re
from pathlib import Path

# Prefer the already created df_raw (from the loader cell)
if 'df_raw' in globals():
    df = globals()['df_raw'].copy()
else:
    # Zoek globale dataframes zoals df_YYYY
    candidate_dfs = [v for k, v in globals().items() if re.match(r'^df_\d{4}$', k) and isinstance(v, pd.DataFrame)]
    if candidate_dfs:
        df = pd.concat(candidate_dfs, ignore_index=True)
    else:
        # Fallback: lees alle CSVs uit Data/Demand of ../data/demand
        data_dir = Path('Data/Demand')
        if not data_dir.exists():
            data_dir = Path('../data/demand')
        if not data_dir.exists():
            data_dir = Path('./data/demand')
        csv_files = sorted(data_dir.glob('*.csv'))
        if not csv_files:
            raise FileNotFoundError(f"No CSV files found in {data_dir.resolve()}")
        df_list = [pd.read_csv(f) for f in csv_files]
        df = pd.concat(df_list, ignore_index=True)

print(f"✅ Combined dataframe: {df.shape[0]} rows x {df.shape[1]} columns")

# Try to detect date column (settlement_date case-insensitive) and show range if possible
date_col = None
for c in df.columns:
    if 'settlement' in c.lower() and 'date' in c.lower():
        date_col = c
        break
if date_col is None:
    # fallback: any column name containing 'date'
    date_candidates = [c for c in df.columns if 'date' in c.lower()]
    date_col = date_candidates[0] if date_candidates else None

if date_col is not None:
    try:
        date_series = pd.to_datetime(df[date_col], errors='coerce')
        print(f"\nDate range: {date_series.min()} to {date_series.max()} (column: {date_col})")
    except Exception:
        print(f"\nDate column detected: {date_col} (could not convert to datetime for range)")
else:
    print("\nGeen date-kolom gevonden om datum-range te tonen.")


✅ Combined dataframe: 434014 rows x 22 columns

Date range: 2001-01-01 00:00:00 to 2025-10-03 00:00:00 (column: SETTLEMENT_DATE)


We verifiëren dat de data correct is ingeladen door de dimensies en het datumbereik te controleren. De dataset bevat 22 kolommen met verschillende metingen: van nationale vraag (ND) tot embedded generatie en interconnector flows. De settlement_date kolom toont aan dat we data hebben van begin 2001 tot oktober 2025.

## 2. Data Cleaning

Om de data toegankelijker te maken en consistentie te waarborgen, passen we een standaard naming convention toe: alle kolomnamen worden lowercase en spaties worden vervangen door underscores. Dit maakt de code leesbaarder en voorkomt potentiële errors bij het refereren naar kolommen.

### 2.1 Kolommen hernoemen

In [4]:
# REQUIREMENT 1: Rename columns to lowercase and remove spaces
df.columns = df.columns.str.lower().str.replace(' ', '_')

print("✅ Nieuwe kolommen:")
print(df.columns.tolist())

✅ Nieuwe kolommen:
['settlement_date', 'settlement_period', 'nd', 'tsd', 'england_wales_demand', 'embedded_wind_generation', 'embedded_wind_capacity', 'embedded_solar_generation', 'embedded_solar_capacity', 'non_bm_stor', 'pump_storage_pumping', 'scottish_transfer', 'ifa_flow', 'ifa2_flow', 'britned_flow', 'moyle_flow', 'east_west_flow', 'nemo_flow', 'nsl_flow', 'eleclink_flow', 'viking_flow', 'greenlink_flow']


Alle 22 kolommen zijn nu consistent hernoemd naar lowercase met underscores. Dit omvat belangrijke variabelen zoals `ND` (de target die we willen voorspellen), embedded renewable generation (`embedded_wind_generation`, `embedded_solar_generation`), en verschillende interconnector flows die energie import/export met buurlanden weergeven.

### 2.2 Date & Time parsing

In [5]:
# Opmerking: sommige CSV's gebruiken e.g. "2009-01-01", andere "01-JAN-2009" -> we proberen meerdere formats en rapporteren wat we vonden.
date_col = None
for c in df.columns:
    if 'settlement' in c.lower() and 'date' in c.lower():
        date_col = c
        break
if date_col is None:
    raise KeyError("Geen settlement_date kolom gevonden in dataframe.")

s = df[date_col].astype(str).str.strip()

# Te proberen formats (voorkomen van common cases incl. '01-JAN-2009')
formats = [
    "%Y-%m-%d",
    "%Y/%m/%d",
    "%d-%b-%Y",   # 01-JAN-2009
    "%d-%B-%Y",   # 01-January-2009
    "%d/%m/%Y",
    "%d.%m.%Y",
    "%d-%m-%Y",
    "%Y%m%d",
]

parsed = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")
format_counts = {}

# Try each explicit format
for fmt in formats:
    mask = parsed.isna() & s.notna()
    if not mask.any():
        break
    try:
        parsed_try = pd.to_datetime(s[mask], format=fmt, errors="coerce")
    except Exception:
        parsed_try = pd.to_datetime(s[mask].replace(" ", ""), format=fmt, errors="coerce")
    parsed.loc[mask] = parsed_try
    count = parsed_try.notna().sum()
    format_counts[fmt] = count

# Laatste poging: inferentie (flexibeler, maar trager)
mask = parsed.isna() & s.notna()
if mask.any():
    inferred = pd.to_datetime(s[mask], infer_datetime_format=True, dayfirst=True, errors="coerce")
    parsed.loc[mask] = inferred
    format_counts["inferred"] = inferred.notna().sum()

total = len(s)
parsed_count = parsed.notna().sum()
unparsed_count = total - parsed_count

print(f"✅ settlement_date parsing summary (column: {date_col}):")
for k, v in format_counts.items():
    print(f"  - parsed with {k}: {v} rows")
print(f"  - totaal parsed: {parsed_count} / {total}")
print(f"  - niet geparsed (NaT): {unparsed_count}")

if unparsed_count > 0:
    sample_bad = s[parsed.isna()].drop_duplicates().tolist()[:10]
    print("\n⚠️ Voorbeelden van ongeldige / onbekende datumformaten (max 10):")
    for val in sample_bad:
        print(f"   - {val!r}")

# Assign parsed datetimes back to dataframe
df[date_col] = parsed

# Create additional time features (useful for modeling)
df['year'] = df[date_col].dt.year
df['month'] = df[date_col].dt.month
df['day'] = df[date_col].dt.day
df['dayofweek'] = df[date_col].dt.dayofweek  # 0=Monday, 6=Sunday
df['quarter'] = df[date_col].dt.quarter
df['week'] = df[date_col].dt.isocalendar().week

# Settlement period is 30-min blocks (1-48) — bescherm tegen missing/incorrect values
if 'settlement_period' in df.columns:
    df['settlement_period'] = pd.to_numeric(df['settlement_period'], errors='coerce')
    df['hour'] = ((df['settlement_period'] - 1) // 2).astype('Int64')
else:
    df['hour'] = pd.NA

print("✅ Time features created:")
print(df[[date_col, 'settlement_period', 'year', 'month', 'day', 'dayofweek', 'hour']].head())


✅ settlement_date parsing summary (column: settlement_date):
  - parsed with %Y-%m-%d: 153502 rows
  - parsed with %Y/%m/%d: 0 rows
  - parsed with %d-%b-%Y: 262992 rows
  - parsed with %d-%B-%Y: 0 rows
  - parsed with %d/%m/%Y: 0 rows
  - parsed with %d.%m.%Y: 0 rows
  - parsed with %d-%m-%Y: 0 rows
  - parsed with %Y%m%d: 0 rows
  - parsed with inferred: 17520 rows
  - totaal parsed: 434014 / 434014
  - niet geparsed (NaT): 0
✅ Time features created:
  settlement_date  settlement_period  year  month  day  dayofweek  hour
0      2001-01-01                  1  2001      1    1          0     0
1      2001-01-01                  2  2001      1    1          0     0
2      2001-01-01                  3  2001      1    1          0     1
3      2001-01-01                  4  2001      1    1          0     1
4      2001-01-01                  5  2001      1    1          0     2


De settlement_date kolom bevat verschillende datumformaten doorheen de jaren omdat de databron kennelijk van format is veranderd. We hebben 153502 rijen met het %Y-%m-%d format (bijv. "2001-01-01") en 262992 rijen met het %d-%b-%Y format (bijv. "01-Jan-2001"). Door deze flexibele parsing aanpak kunnen we alle datums correct converteren naar datetime objecten. Daarnaast extraheren we ook nuttige temporele features zoals year, month, dayofweek, hour en quarter die later belangrijk zullen zijn voor onze modellen om seizoenspatronen en trends te detecteren.

### 2.3 Missing Values Analysis

In [6]:
# REQUIREMENT 2: Look for NA values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing_Count': missing.values,
    'Missing_Percentage': missing_pct.values
})

missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

print("⚠️ Kolommen met missing values:")
display(missing_df)

print(f"\n📊 Totaal aantal kolommen met missing data: {len(missing_df)}")

⚠️ Kolommen met missing values:


,Column,Missing_Count,Missing_Percentage
11,scottish_transfer,385680,88.863493
18,nsl_flow,315552,72.705489
19,eleclink_flow,315552,72.705489
20,viking_flow,315552,72.705489
21,greenlink_flow,315552,72.705489
7,embedded_solar_generation,140256,32.316008
8,embedded_solar_capacity,140256,32.316008
13,ifa2_flow,140256,32.316008
14,britned_flow,140256,32.316008
16,east_west_flow,140256,32.316008



📊 Totaal aantal kolommen met missing data: 15


De analyse toont dat verschillende kolommen substantiële hoeveelheden missende data bevatten, maar dit is verwacht en heeft een duidelijke oorzaak: deze features zijn pas later toegevoegd aan het meetsysteem. 

**Belangrijkste observaties:**
- `scottish_transfer` heeft 89% missende waarden omdat deze interconnector pas later operationeel werd
- Nieuwe interconnectors zoals `nsl_flow`, `eleclink_flow`, `viking_flow` en `greenlink_flow` (allen ~73% missing) zijn recent toegevoegd aan het energienetwerk
- Embedded renewable data (`embedded_wind_generation`, `embedded_solar_generation` met 24-32% missing) werd pas vanaf een bepaald jaar systematisch bijgehouden naarmate deze technologieën mainstream werden
- `tsd` (Transmission System Demand) en enkele andere flows hebben ook missing values uit vroegere jaren

Deze missende waarden zijn geen data quality issues maar reflecteren de historische ontwikkeling van het UK energiesysteem. We zullen deze opvullen met 0, wat correct is omdat de afwezigheid van data betekent dat die capaciteit of connectie toen nog niet bestond.

### 2.4 Missing values

In [7]:
df_clean = df.copy()
df_clean = df_clean.fillna(0)

print("✅ Missing values na opvullen met 0:", df_clean.isnull().sum().sum())

✅ Missing values na opvullen met 0: 0


Door alle missende waarden met 0 op te vullen hebben we nu een complete dataset van 434014 rijen zonder missing values. Deze imputation strategie is gerechtvaardigd omdat:
1. Missende interconnector flows betekenen dat die verbinding nog niet operationeel was (dus effectief 0 MW flow)
2. Missende embedded generation data betekent dat die capaciteit nog niet gemeten werd of verwaarloosbaar klein was
3. Dit voorkomt dat we waardevolle historische data moeten weggooien

Het alternatief (rijen met missing values verwijderen) zou betekenen dat we vooral recente data zouden overhouden, wat onze mogelijkheid om langetermijntrends te analyseren zou beperken.

### 2.5 Categorical data

In [8]:
categorical_features = ['month', 'dayofweek', 'hour', 'quarter', 'settlement_period']

for col in categorical_features:
    df_clean[col] = df_clean[col].astype('category')

print("✅ Categorical features:")
print(df_clean[categorical_features].dtypes)

✅ Categorical features:
month                category
dayofweek            category
hour                 category
quarter              category
settlement_period    category
dtype: object


We converteren temporele features naar categorische variabelen omdat ze cyclische patronen vertegenwoordigen. Een machine learning model moet begrijpen dat maand 12 (december) en maand 1 (januari) dichter bij elkaar liggen dan december en juni, ondanks de numerieke waarden. Door deze als categorieën te behandelen kunnen tree-based modellen zoals XGBoost deze cyclische patronen beter leren. Dit geldt vooral voor:
- `month`: seizoenspatronen (winter vs zomer verbruik)
- `dayofweek`: weekpatronen (werkdagen vs weekend)  
- `hour`: dagelijkse patronen (piekuren vs daluren)
- `settlement_period`: de specifieke half-uurlijkse periode binnen een dag

In [9]:
# Create readable labels voor dayofweek
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
             4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df_clean['day_name'] = df_clean['dayofweek'].map(day_names)

# Is weekend?
df_clean['is_weekend'] = df_clean['dayofweek'].isin([5, 6]).astype(int)

print("✅ Extra features created:")
print(df_clean[['dayofweek', 'day_name', 'is_weekend']].head(10))

✅ Extra features created:
  dayofweek day_name  is_weekend
0         0   Monday           0
1         0   Monday           0
2         0   Monday           0
3         0   Monday           0
4         0   Monday           0
5         0   Monday           0
6         0   Monday           0
7         0   Monday           0
8         0   Monday           0
9         0   Monday           0


We hebben twee nuttige afgeleide features toegevoegd:
- **day_name**: Een leesbare versie van de dag (Monday, Tuesday, etc.) die visualisaties intuïtiever maakt
- **is_weekend**: Een binaire indicator (0/1) die het onderscheid markeert tussen weekdagen en weekenden

Deze weekend feature is bijzonder waardevol omdat elektriciteitsverbruik significant verschilt tussen werk- en weekenddagen. Op werkdagen zien we hogere pieken door kantoren, winkels en industrie, terwijl weekenden een vlakker verbruikspatroon tonen. Dit patroon is ook zichtbaar in de Christmas Day grafieken die we eerder lieten zien, waar een feestdag zich gedraagt als een weekend qua verbruiksprofiel.

De dataset is nu volledig gereinigd en verrijkt met temporele features. In de volgende notebooks zullen we deze data gebruiken voor exploratory data analysis en het bouwen van voorspellingsmodellen.